## **Installing Libraries**

In [ ]:
pip install transformers[torch]

In [ ]:
pip install ipywidgets

## **Import all necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import Trainer
import torch
import torch.nn.functional as F
import wandb 
from kaggle_secrets import UserSecretsClient
from huggingface_hub import notebook_login




In [ ]:

user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("wandb_key")
wandb.login(key=secret_value)

## **Loading the clean dataset**

In [ ]:

news=pd.read_csv('/kaggle/input/fakenews/clean.csv')
news.head()

## **Loading the base model and tokenizer**

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base")

In [ ]:


tokenizer = AutoTokenizer.from_pretrained("distilroberta-base",clean_up_tokenization_spaces=True)

In [ ]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## **Tokenizing the dataset**

In [ ]:

seed=18
features = news['clean_text'].tolist()
targets = news['class'].tolist()

# splitting data into training, testing, and validation sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.20, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=seed)

In [ ]:
train_encodings = tokenizer(X_train,max_length = tokenizer.model_max_length,truncation=True,padding=True,return_tensors='pt')
test_encodings = tokenizer(X_test,max_length = tokenizer.model_max_length,truncation=True,padding=True,return_tensors='pt')
val_encodings = tokenizer(X_val,max_length = tokenizer.model_max_length,truncation=True,padding=True,return_tensors='pt')


## **Converting the dataset to Pytorch format**

In [ ]:

class NewsDataset(Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    # convert to pytorch tensors
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item 
    
    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)
val_dataset = NewsDataset(val_encodings, y_val)

## **Fine tuning the model**

In [ ]:

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    report_to=None
)

In [ ]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    acc = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    
    return {"acc": acc, "precision": precision, "recall": recall, "f1 score": f1}



In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

## **Evaluation of model**

In [ ]:
trainer.evaluate(test_dataset)

## **Save the model**

In [ ]:
trainer.save_model('fake_detector')

## **Push the model to hugging face**

In [ ]:
notebook_login()

In [ ]:
trainer.push_to_hub("Am09/distilroberta-base-fake_news_detector-Am09") 